<a href="https://colab.research.google.com/github/Nadia147/Machine_learning-Deep_learning/blob/main/FINAL_2d3d_PU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install earthpy
# Colab Requirements
!pip install spectral
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
## Basics
import gc
gc.collect()
import warnings
warnings.filterwarnings('ignore')
import time
import numpy
import numpy as np
from PIL import Image
from operator import truediv
import scipy.io as sio
import os
import pandas as pd
import seaborn as sns
import spectral
## Ploting
from plotly.offline import init_notebook_mode
import plotly.graph_objs as go
from matplotlib.pyplot import cm
import matplotlib.pyplot as plt
init_notebook_mode(connected=True)
%matplotlib inline
## Sklearn
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, cohen_kappa_score
from sklearn.decomposition import IncrementalPCA
## Deep Model
import keras
import h5py
from keras.layers import Dropout, Input, Conv2D, Conv3D, MaxPool3D, Flatten, Dense, Reshape, BatchNormalization
from plotly.offline import iplot, init_notebook_mode
from keras.losses import categorical_crossentropy
from keras.optimizers import Adadelta
from keras.models import Sequential, Model
from keras.utils import np_utils
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
## Mounting Colab
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')
import matplotlib.pyplot as plt

import numpy as np
import earthpy.plot as ep
import seaborn as sns
import earthpy.spatial as es

import plotly.graph_objects as go
import plotly.express as px

from scipy.io import loadmat

import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import (accuracy_score,
                             confusion_matrix, classification_report)

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from tqdm import tqdm
import scipy.io

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.9/212.9 kB 3.1 MB/s eta 0:00:00


Instructions for updating:
non-resource variables are not supported in the long term


Mounted at /content/drive


In [ ]:
## Split Training, Validation and Test Sets
def SplitTr_Te(HSI, GT, TeRatio, randomState=345):
    Tr, Te, TrC, TeC = train_test_split(HSI, GT, test_size=TeRatio, random_state=randomState, stratify=GT)
    return Tr, Te, TrC, TeC
def DL_Method(HSI, numComponents = 75):
    RHSI = np.reshape(HSI, (-1, HSI.shape[2]))
    n_batches = 256
    inc_pca = IncrementalPCA(n_components=numComponents)
    for X_batch in np.array_split(RHSI, n_batches):
        inc_pca.partial_fit(X_batch)
    X_ipca = inc_pca.transform(RHSI)
    RHSI = np.reshape(X_ipca, (HSI.shape[0],HSI.shape[1], numComponents))
    return RHSI
## Padding
def ZeroPad(HSI, margin=2):
    NHSI = np.zeros((HSI.shape[0] + 2 * margin, HSI.shape[1] + 2* margin, HSI.shape[2]))
    x_offset = margin
    y_offset = margin
    NHSI[x_offset:HSI.shape[0] + x_offset, y_offset:HSI.shape[1] + y_offset, :] = HSI
    return NHSI
## Spatial Patches in 3D
def HSICubes(HSI, GT, WinSize=5, removeZeroLabels = True):#5t01
    margin = int((WinSize - 1) / 2)
    zeroPaddedX = ZeroPad(HSI, margin=margin)
    # split patches
    patchesData = np.zeros((HSI.shape[0] * HSI.shape[1], WinSize, WinSize, HSI.shape[2]))
    patchesLabels = np.zeros((HSI.shape[0] * HSI.shape[1]))
    patchIndex = 0
    for r in range(margin, zeroPaddedX.shape[0] - margin):
        for c in range(margin, zeroPaddedX.shape[1] - margin):
            patch = zeroPaddedX[r - margin:r + margin + 1, c - margin:c + margin + 1]
            patchesData[patchIndex, :, :, :] = patch
            patchesLabels[patchIndex] = GT[r-margin, c-margin]
            patchIndex = patchIndex + 1
    if removeZeroLabels:
        patchesData = patchesData[patchesLabels>0,:,:,:]
        patchesLabels = patchesLabels[patchesLabels>0]
        patchesLabels -= 1
    return patchesData, patchesLabels
## Compute Per Class Accuacy form Confusion Matrix
def AA_andEachClassAccuracy(confusion_matrix):
    counter = confusion_matrix.shape[0]
    list_diag = np.diag(confusion_matrix)
    list_raw_sum = np.sum(confusion_matrix, axis=1)
    each_acc = np.nan_to_num(truediv(list_diag, list_raw_sum))
    average_acc = np.mean(each_acc)
    return each_acc, average_acc



import time
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, cohen_kappa_score

def evaluate_performance(model, test_data, test_labels):
    start = time.time()
    Y_pred = model.predict(test_data)
    y_pred = np.argmax(Y_pred, axis=1)
    end = time.time()
    total_time = end - start

    target_names = ['Asphalt', 'Meadows', 'Gravel', 'Trees', 'Metal Sheet',
                    'Bare Soil', 'Bitumen', 'Brick', 'Shadow']

    classification = classification_report(np.argmax(test_labels, axis=1), y_pred, target_names=target_names)
    oa = accuracy_score(np.argmax(test_labels, axis=1), y_pred)
    confusion = confusion_matrix(np.argmax(test_labels, axis=1), y_pred)
    each_acc, aa = AA_andEachClassAccuracy(confusion)  # You'll need to define AA_andEachClassAccuracy function

    kappa = cohen_kappa_score(np.argmax(test_labels, axis=1), y_pred)
    score = model.evaluate(test_data, test_labels, batch_size=32)
    test_loss = score[0] * 100
    test_accuracy = score[1] * 100

    return classification, confusion, test_loss, test_accuracy, oa * 100, each_acc * 100, aa * 100, kappa * 100, target_names, y_pred, total_time
## Assigning Class Labels and Computing the Test Accuracy
def reports(Te,TeC):
    start = time.time()
    Y_pred = model.predict(Te)
    y_pred = np.argmax(Y_pred, axis=1)
    end = time.time()
    total = end - start
    target_names = ['Asphalt', 'Meadows', 'Gravel', 'Trees', 'Metal Sheet',
                    'Bare Soil', 'Bitumen', 'Brick', 'Shadow']
    classification = classification_report(np.argmax(TeC, axis=1), y_pred, target_names=target_names)
    oa = accuracy_score(np.argmax(TeC, axis=1), y_pred)
    confusion = confusion_matrix(np.argmax(TeC, axis=1), y_pred)
    each_acc, aa = AA_andEachClassAccuracy(confusion)
    kappa = cohen_kappa_score(np.argmax(TeC, axis=1), y_pred)
    score = model.evaluate(Te, TeC, batch_size=32)
    Test_Loss =  score[0]*100
    Test_accuracy = score[1]*100
    return classification, confusion, Test_Loss, Test_accuracy, oa*100, each_acc*100, aa*100, kappa*100, target_names, y_pred, total

## Compute the Patch to Prepare for Ground Truths
def Patch(data,height_index,width_index):
    height_slice = slice(height_index, height_index+PATCH_SIZE)
    width_slice = slice(width_index, width_index+PATCH_SIZE)
    patch = data[height_slice, width_slice, :]
    return patch

In [ ]:

#ACCURACY BARSE

import scipy.io
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
from keras.layers import Input, Conv2D, Conv3D, Flatten, Dense, Dropout
from keras.models import Model
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import time
from keras import backend as K

# Load Pavia University dataset
HSI = scipy.io.loadmat('/content/drive/MyDrive/Colab Notebooks/PaviaU.mat')['paviaU']
GT = scipy.io.loadmat('/content/drive/MyDrive/Colab Notebooks/PaviaU_gt.mat')['paviaU_gt']

# Reduce Dimensionality (Assuming DL_Method and HSICubes functions are defined)
HSI = DL_Method(HSI, numComponents=20)
HSI, GT = HSICubes(HSI, GT, WinSize=11)

# Split Train and Test sets
Tr, Te, TrC, TeC = train_test_split(HSI, GT, test_size=0.60, random_state=42)

# Split Train and Validation
Tr, Tv, TrC, TvC = train_test_split(Tr, TrC, test_size=0.30, random_state=42)

# Reshape and preprocess data
Tr = Tr.reshape(-1, 11, 11, 20, 1)
TrC = np_utils.to_categorical(TrC)
Tv = Tv.reshape(-1, 11, 11, 20, 1)
TvC = np_utils.to_categorical(TvC)

# Model Structure
input_layer = Input((11, 11, 20, 1))

# 2D Convolutional Layer
conv_layer2d = Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(input_layer)

# Reshape the 2D output to 3D using Keras backend
reshaped_layer = K.expand_dims(conv_layer2d, axis=-1)

# 3D Convolutional Layers
conv_layer3d_1 = Conv3D(filters=8, kernel_size=(3, 3, 7), activation='relu')(reshaped_layer)
conv_layer3d_2 = Conv3D(filters=16, kernel_size=(3, 3, 5), activation='relu')(conv_layer3d_1)
conv_layer3d_3 = Conv3D(filters=32, kernel_size=(3, 3, 3), activation='relu')(conv_layer3d_2)
conv_layer3d_4 = Conv3D(filters=64, kernel_size=(3, 3, 3), activation='relu')(conv_layer3d_3)

# Flatten Layer
flatten_layer = Flatten()(conv_layer3d_4)

# Fully Connected Layers
dense_layer1 = Dense(units=256, activation='relu')(flatten_layer)
dense_layer1 = Dropout(0.4)(dense_layer1)
dense_layer2 = Dense(units=128, activation='relu')(dense_layer1)
dense_layer2 = Dropout(0.4)(dense_layer2)

# Output Layer
output_layer = Dense(units=9, activation='softmax')(dense_layer2)

# Define the model
model = Model(inputs=input_layer, outputs=output_layer)
model.summary()

# Compile the model
adam = Adam(lr=0.001, decay=1e-06)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

# Training and Fit the Model
start = time.time()
history = model.fit(x=Tr, y=TrC, batch_size=256, epochs=50, validation_data=(Tv, TvC))
end = time.time()
Tr_Time = end - start
print(Tr_Time)


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 11, 11, 20, 1)]   0         
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 9, 18, 32)     320       
                                                                 
 tf_op_layer_ExpandDims (Ten  [(None, 11, 9, 18, 32, 1  0        
 sorFlowOpLayer)             )]                                  
                                                                 
 conv3d_1 (Conv3D)           (None, 11, 7, 16, 26, 8)  512       
                                                                 
 conv3d_2 (Conv3D)           (None, 11, 5, 14, 22, 16  5776      
                             )                                   
                                                                 
 conv3d_3 (Conv3D)           (None, 11, 3, 12, 20, 32  13856 

In [ ]:
import scipy.io
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
from keras.layers import Input, Conv2D, Conv3D, Flatten, Dense, Dropout
from keras.models import Model
from keras.optimizers import Adam
import time
from keras import backend as K

# Load Pavia University dataset
HSI = scipy.io.loadmat('/content/drive/MyDrive/Colab Notebooks/PaviaU.mat')['paviaU']
GT = scipy.io.loadmat('/content/drive/MyDrive/Colab Notebooks/PaviaU_gt.mat')['paviaU_gt']

# Assuming DL_Method and HSICubes functions are defined
HSI = DL_Method(HSI, numComponents=20)
HSI, GT = HSICubes(HSI, GT, WinSize=11)

# Data splitting
Tr, Tv, TrC, TvC = train_test_split(HSI, GT, test_size=0.40, random_state=42)
Tr, Tv, TrC, TvC = train_test_split(Tr, TrC, test_size=0.30, random_state=42)

# Data preprocessing
Tr, Tv = Tr.reshape(-1, 11, 11, 20, 1), Tv.reshape(-1, 11, 11, 20, 1)
TrC, TvC = np_utils.to_categorical(TrC), np_utils.to_categorical(TvC)

# Model definition
input_layer = Input((11, 11, 20, 1))
conv_layer2d = Conv2D(32, (3, 3), activation='relu')(input_layer)
reshaped_layer = K.expand_dims(conv_layer2d, -1)
#conv_layer3d = [Conv3D(filters, (3, 3, k), activation='relu')(conv_layer3d) for filters, k in zip([8, 16, 32, 64], [7, 5, 3, 3])]
flatten_layer = Flatten()(conv_layer3d[-1])
dense_layer = [Dense(units, activation='relu')(flatten_layer) for units in [256, 128]]
dense_layer = [Dropout(0.4)(dense) for dense in dense_layer]
output_layer = Dense(9, activation='softmax')(dense_layer[-1])
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001, decay=1e-06), metrics=['accuracy'])
model.summary()
# Model training
start = time.time()
history = model.fit(x=Tr, y=TrC, batch_size=256, epochs=50, validation_data=(Tv, TvC))
end = time.time()
Tr_Time = end - start
print(Tr_Time)


NameError: ignored

In [ ]:
#98.65%
!pip install matplotlib==2.*

In [ ]:
## Plot Training and Validation loss and Accuracy
plt.figure(figsize=(7,7))
plt.rcParams.update({'font.size': 14})
plt.grid()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.title('Training and Validation loss')
plt.legend(['Training','Validation'], loc='upper right')
#plt.savefig("IP_loss.eps")
plt.show()
## Plot Training and Validation Accuracy
plt.figure(figsize=(7,7))
plt.ylim(0,1.1)
plt.grid()
plt.plot(history.history['acc'])
#plt.plot(history.history['accuracy'])
#plt.plot(history.history['val_accuracy'])
plt.plot(history.history['val_acc'])
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.title('Training and Validation Accuracy')
plt.legend(['Training','Validation'])
#plt.savefig("IP_Accuracy.eps")
plt.show()
## Testing Phase
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['acc'])
## Reshape Test Data
Te = Te.reshape(-1, 11, 11, 20, 1)
TeC = np_utils.to_categorical(TeC)
Te.shape, TeC.shape
## Computing and Writing the Accuacy in .txt file
classification,confusion,Test_loss,Test_accuracy,oa,each_acc,aa,kappa,target_names,y_pred,Time = reports(Te,TeC)
classification = str(classification)
confusion = str(confusion)
print(classification), print(Test_accuracy), print(oa), print(aa), print(Tr_Time), print(Time)
## Draw Confusion Matrix
confusion = confusion_matrix(np.argmax(TeC, axis=1), y_pred, labels=np.unique(np.argmax(TeC, axis=1)))
cm_sum = np.sum(confusion, axis=1, keepdims=True)
cm_perc = confusion / cm_sum.astype(float) * 100
annot = np.empty_like(confusion).astype(str)
nrows, ncols = confusion.shape
for i in range(nrows):
    for j in range(ncols):
        c = confusion[i, j]
        p = cm_perc[i, j]
        if i == j:
            s = cm_sum[i]
            annot[i, j] = '%.1f%%\n%d/%d' % (p, c, s)
        elif c == 0:
            annot[i, j] = ''
        else:
            annot[i, j] = '%.1f%%\n%d' % (p, c)
cm = pd.DataFrame(confusion, index=np.unique(target_names), columns=np.unique(target_names))
cm.index.name = 'Actual'
cm.columns.name = 'Predicted'
fig, ax = plt.subplots(figsize=(15,10))
plt.rcParams.update({'font.size': 12})
sns.heatmap(cm, cmap= "YlGnBu", annot=annot, fmt='', ax=ax)
## Loading Dataset
data_path = os.path.join(os.getcwd(),'/content/drive/My Drive/Colab')
HSI = scipy.io.loadmat('/content/drive/MyDrive/Colab Notebooks/PaviaU.mat')['paviaU']
GT = scipy.io.loadmat('/content/drive/MyDrive/Colab Notebooks/PaviaU_gt.mat')['paviaU_gt']

## Check the Dimensions of HSI
height = GT.shape[0]
width = GT.shape[1]
PATCH_SIZE = 11
## Dimensional Reduction and zero padding
HSI = DL_Method(HSI, numComponents=20)
HSI.shape
HSI = ZeroPad(HSI, 11//2)
## Calculate the predicted Ground Truths
outputs = np.zeros((height,width))
for i in range(height):
    for j in range(width):
        target = int(GT[i,j])
        if target == 0 :
            continue
        else :
            image_patch=Patch(HSI,i,j)
            X_test_image = image_patch.reshape(1,image_patch.shape[0],
                                               image_patch.shape[1], image_patch.shape[2], 1).astype('float32')
            prediction = (model.predict(X_test_image))
            prediction = np.argmax(prediction, axis=1)
            outputs[i][j] = prediction+1
## Show Ground Truths
ground_truth = spectral.imshow(classes = GT,figsize =(7,7))
predict_image = spectral.imshow(classes = outputs.astype(int),figsize =(7,7))
spectral.save_rgb("predictions.png", outputs.astype(int), colors=spectral.spy_colors)
spectral.save_rgb("ground_truth.png", GT, colors=spectral.spy_colors)

In [ ]:
# Calculate and Print Average Accuracy
average_accuracy = np.mean(history.history['val_acc'])
print(f"Average Validation Accuracy: {average_accuracy:.4f}")
y_pred = model.predict(Tv)
y_pred_labels = np.argmax(y_pred, axis=1)
true_labels = np.argmax(TvC, axis=1)

kappa = cohen_kappa_score(true_labels, y_pred_labels)
print(f"Kappa Score: {kappa:.4f}")

y_pred = model.predict(Tv)
y_pred_labels = np.argmax(y_pred, axis=1)
true_labels = np.argmax(TvC, axis=1)

accuracy = accuracy_score(true_labels, y_pred_labels)
print(f"Overall Accuracy: {accuracy:.4f}")